# Bayesian linear regression ☕️☕️☕️☕️

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Être sensibilisé aux liens entre approche Bayésienne et&nbsp;:
    * régularisation,
    * ensemble de modèles.
* Être capable&nbsp;:
    * d'utiliser la régression linéaire Bayesienne dans $\texttt{sklearn}$.
    


 ----

Vous trouverez un approfondissement non nécessaire à cette séquence et traitant de l'inférence Bayésienne à l'adresse&nbsp;:

* [Inférence Bayésienne](https://github.com/maximiliense/lmiprp/raw/main/Cours/Inférence%20Bayésienne/cours.pdf)

## I. Posterior prédictive et modèle

Comme nous avons pu le voir, le principe du *Bayesian Model Averaging* se généralise bien sûr lorsqu'on a une quantité infinie de modèles où la *posterior* prédictive s'écrit :

$$p(y|x_\text{new}, S_n)=\int p(y|x_\text{new}, \mathcal{M}, S_n)p(\mathcal{M}|S_n)dM.$$

Nous pouvons formaliser une version Bayésienne de la régression linéaire en suivant ce principe.

### A. Le modèle 

Considérons le modèle linéaire suivant. Soit $X\in\mathbb{R}^d$, $\omega\in\mathbb{R}^d$, $\epsilon\sim\mathcal{N}(0, \sigma^2)$ et considérons la dépendence suivante :

$$Y=\langle \omega, X\rangle + \epsilon.$$

On supposera dans cet exercice que les matrices qu'on inverse sont inversibles.

Nous avons donc $Y\sim\mathcal{N}(\langle\omega, X\rangle, \sigma^2)$. Ici, chaque paramétrisation (chaque choix de $\omega$) peut être vu comme un modèle. Comme pour le cas précédent, nous devons donner une mesure *a priori* pour chacun de nos modèles. Nous choissons ici le *prior* suivant :

$$p(\omega)=\mathcal{N}(0, \sigma^2\Lambda^{-1}),$$

où $\Lambda$ est la matrice de précision. Par facilité, considérons $\Lambda=\lambda I$ et notons $\Sigma=\sigma^2\Lambda^{-1}$.  

## II. Détails des calculs

(Si vous n'êtes intéressés que par la solution, vous pouvez sauter directement les calculs)

### 1. Calcul de la posterior sur $\omega$

Nous cherchons dans un premier temps à calculer :

$$p(y|\tilde{X}, \omega)\propto\text{exp}\Big(-\frac{1}{2\sigma^2}(y-\tilde{X}\omega )^T(y-\tilde{X}\omega)\Big)$$

où

$$\tilde{X}=\begin{pmatrix}x_1^T\\\vdots\\x_n^T\end{pmatrix}\text{ et }y=\begin{pmatrix}y_1\\\vdots\\y_n\end{pmatrix}.$$

C'est notre likelihood. Si en maximisant cette quantité qu'on obtient (lorsque $\tilde{X}^T\tilde{X}$ est inversible) :

$$\hat{\omega}=(\tilde{X}^T\tilde{X})^{-1}\tilde{X}^ty$$

On rappelle notre prior sur $\omega$ :

$$p(\omega)\propto\text{exp}\Big(-\frac{1}{2}\omega^T\Sigma^{-1}\omega\Big)=\text{exp}\Big(-\frac{\lambda}{2\sigma^2}\omega^T\omega\Big).$$

Nous avons donc tous les éléments nous permettant de calculer notre *posterior* :

$$p(\omega|\tilde{X}, y)\propto p(y|\tilde{X},\omega)p(\omega).$$

En remplaçant les quantités ci-dessus, nous obtenons donc :

$$p(\omega|\tilde{X}, y)\propto \text{exp}\Big(-\frac{1}{2\sigma^2}(y-\tilde{X}\omega )^T(y-\tilde{X}\omega)-\frac{\lambda}{2\sigma^2}\omega^T\omega\Big).$$

Notre prior sur $\omega$ est conjugué avec notre vraisemblance. Cela veut dire que la *posterior* est donc nécessairement une loi normale également. Ainsi, on sait qu'elle sera proportionnelle à l'exponentielle de :

$$(\omega-\mu)^T{\Sigma^\prime}^{-1}(\omega-\mu)=\omega^T{\Sigma^\prime}^{-1}\omega-2\omega^T{\Sigma^\prime}^{-1}\mu+\text{constante}.$$

où $\mu$ sera le paramètre de moyenne et $\Sigma^\prime$ notre nouvelle matrice de covariance. Considérons donc la quantité $\frac{1}{\sigma^2}(y-\tilde{X}\omega )^T(y-\tilde{X}\omega)+\frac{\lambda}{\sigma^2}\omega^T\omega$ (nous avons factorisé par $-1/2$ les éléments de l'exponentiel) et cherchons à obtenir la forme classique attendue d'une loi normale :

$$\begin{aligned}
\frac{1}{\sigma^2}(y-\tilde{X}\omega )^T(y-\tilde{X}\omega)+\frac{\lambda}{\sigma^2}\omega^T\omega&=\sigma^{-2}(y^Ty-2\omega^T\tilde{X}^T y+\omega^T\tilde{X}^T\tilde{X}\omega)+\lambda\sigma^{-2}\omega^T\omega\\
&=\sigma^{-2}y^Ty-2\sigma^{-2}\omega^T\tilde{X}^Ty+\sigma^{-2}\omega^T(\tilde{X}^T\tilde{X}+\lambda I)\omega
\end{aligned}$$

On remarque ainsi directement que :

$$\begin{aligned}
{\Sigma^\prime}^{-1}&=\frac{\tilde{X}^T\tilde{X}+\lambda I}{\sigma^2}
\end{aligned}$$

Et on en déduit que :

$$\mu=(\tilde{X}^T\tilde{X}+\lambda I)^{-1}\tilde{X}^T y.$$

On remarque que l'espérance de notre posterior est exactement l'estimateur de Ridge. Dans notre cas de figure, la posterior est une loi normale et est ainsi symétrique : son maximum est atteint par son espérance. On parle souvent de MAP, ou Maximum A Posteriori.

### 2. Calcul de la posterior prédictive sur $y_\text{new}$

Maintenant que nous avons la posterior pour nos modèles, nous pouvons calculer la *posterior* prédictive. Celle-ci s'exprime de la manière suivante :

$$\begin{aligned}
p(y_{\text{new}}| x_{\text{new}}, \tilde{X}, y)&=\int_\omega p(y_{\text{new}}|x_{\text{new}}, \omega)p(\omega|\tilde{X}, y)d\omega\text{ (la dépendance en $X$ disparait)}\\
&\propto\int_\omega\text{exp}\Big(-\frac{1}{2\sigma^2}(y_{\text{new}}-\langle \omega, x_{\text{new}}\rangle)^2\Big)\text{exp}\Big(-\frac{1}{2}(\omega-\mu)^T{\Sigma^\prime}^{-1}(\omega-\mu)\Big)d\omega\\
&=\int_\omega\text{exp}\Big(-\frac{1}{2}\big(\sigma^{-2}(y_{\text{new}}^2-2y_{\text{new}}\langle \omega, x_{\text{new}}\rangle+(\langle \omega, x_{\text{new}}\rangle)^2+\omega^T{\Sigma^\prime}^{-1}\omega-2\omega^T{\Sigma^\prime}^{-1}\mu+\\&\mu^T{\Sigma^\prime}^{-1}\mu\big)\Big)d\omega
\end{aligned}$$

où $\mu^T{\Sigma^\prime}^{-1}\mu$ n'agit que comme un facteur de proportionnalité et peut donc être "éliminé".


#### Étape 2.1 : Intégration de la variable $\omega$

Nous allons dans un premier temps regrouper tous les éléments faisant intervenir $\omega$, notre variable d'intégration.


$$\begin{aligned}
p(y_{\text{new}}| x_{\text{new}}, \tilde{X}, y)&\propto\int_\omega\text{exp}\Big(-\frac{1}{2}\big(\sigma^{-2}(y_{\text{new}}^2-2y_{\text{new}}\langle \omega, x_{\text{new}}\rangle+(\langle \omega, x_{\text{new}}\rangle)^2+\omega^T{\Sigma^\prime}^{-1}\omega-2\omega^T{\Sigma^\prime}^{-1}\mu\big)\Big)d\omega
\end{aligned}$$


Considérons à nouveau la partie entre parenthèses :

$$\begin{aligned}
\sigma^{-2}(y_{\text{new}}^2-2y_{\text{new}}&\langle \omega, x_{\text{new}}\rangle+(\langle \omega, x_{\text{new}}\rangle)^2+\omega^T{\Sigma^\prime}^{-1}\omega-2\omega^T{\Sigma^\prime}^{-1}\mu\\
&=y_\text{new}^2\sigma^{-2}-2\omega^T({\Sigma^\prime}^{-1}\mu+x_\text{new}y_\text{new}\sigma^{-2})+\omega^T(x_\text{new}x_\text{new}^T\sigma^{-2}+{\Sigma^\prime}^{-1})\omega\\
&=(\omega-\mu^\prime){\Sigma^{\prime\prime}}^{-1}(\omega-\mu^\prime)-{\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime+y_{\text{new}}^2\sigma^{-2}
\end{aligned}$$


où :

$${\Sigma^{\prime\prime}}^{-1}=x_\text{new}x_\text{new}^T\sigma^{-2}+{\Sigma^\prime}^{-1},$$

et :

$$\mu^\prime={\Sigma^{\prime\prime}}({\Sigma^\prime}^{-1}\mu+x_{\text{new}}y_\text{new}\sigma^{-2}).$$

On en déduit ainsi :

$$\begin{aligned}
p(y_{\text{new}}| x_{\text{new}}, \tilde{X}, y)&\propto\text{exp}\Big(\frac{1}{2}{\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime-\frac{1}{2}y_{\text{new}}^2\sigma^{-2}\Big)\int_\omega \text{exp}\Big(-\frac{1}{2}(\omega-\mu^\prime){\Sigma^{\prime\prime}}^{-1}(\omega-\mu^\prime)\Big)d\omega\\
&\propto\text{exp}\Big(\frac{1}{2}{\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime-\frac{1}{2}y_{\text{new}}^2\sigma^{-2}\Big),
\end{aligned}$$

où nous avons pu sortir les éléments constants relativement à $\omega$ de l'intégral. N'y trouvant plus qu'une loi normale dont la probabilité vaut $1$ sur son domaine, nous avons pu éliminer l'intégrale (toujours à un facteur proportionnel près).

#### Étape 2.2 : Calcul de la loi de $y_\text{new}$

On sent que la loi de $y_\text{new}$ sera aussi une loi normale, et on va essayer de retrouver quelque chose qui ressemble à :
On sait qu'on aura quelque chose de la forme :

$$\begin{aligned}
p(y_\text{new}|x_\text{new}, \tilde{X}, y)&\propto\text{exp}\Big(-\frac{1}{2{\sigma^\prime}^2}(y_\text{new}-\mu^{\prime\prime)^2}\Big)\\
&=\text{exp}\Big(-\frac{1}{2}({\sigma^\prime}^{-2}y_\text{new}^2-2{\sigma^\prime}^{-2}y_\text{new}\mu^{\prime\prime}+{\sigma^\prime}^{-2}{\mu^{\prime\prime}}^2)\Big),
\end{aligned}
$$

où $\mu^{\prime\prime}$ sera l'espérance de notre prédiction et $\sigma^\prime$ son écart-type. Rappelons que notre point de départ ici est : 

$$\text{exp}\Big(\frac{1}{2}{\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime-\frac{1}{2}y_{\text{new}}^2\sigma^{-2}\Big).$$

En constatant que $\Sigma, \Sigma^\prime$ ou encore $\Sigma^{\prime\prime}$ sont toutes symétriques, nous avons :

$$\begin{aligned}
{\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime&=({\Sigma^\prime}^{-1}\mu+x_{\text{new}}y_\text{new}\sigma^{-2})^T{\Sigma^{\prime\prime}}{\Sigma^{\prime\prime}}^{-1}{\Sigma^{\prime\prime}}({\Sigma^\prime}^{-1}\mu+x_{\text{new}}y_\text{new}\sigma^{-2})\\
&=\sigma^{-2}y_\text{new}x_\text{new}^T\Sigma^{\prime\prime}\sigma^{-2}y_\text{new}x_\text{new}+2y_\text{new}\sigma^{-2}x_\text{new}^T\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}\mu+\text{constante}\\
&=(\sigma^{-4}x_\text{new}^T\Sigma^{\prime\prime}x_\text{new})y_\text{new}^2+2(\sigma^{-2}x_\text{new}^T\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}\mu) y_\text{new}+\text{constante}.
\end{aligned}$$

En combinant ${\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime$ avec $-y_{\text{new}}^2\sigma^{-2}$, nous obtenons :

$$\begin{aligned}
{\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime-y_{\text{new}}^2\sigma^{-2}&=(\sigma^{-4}x_\text{new}^T\Sigma^{\prime\prime}x_\text{new}-\sigma^{-2})y_\text{new}^2+2(\sigma^{-2}x_\text{new}^T\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}\mu) y_\text{new}+\text{c}
\end{aligned}$$

Remarquons que dans l'exponentielle nous n'avons pas indiqué le signe "-" classique de la loi normale. Nous obtenons ainsi :

$$\begin{aligned}
y_{\text{new}}^2\sigma^{-2}-{\mu^\prime}^T{\Sigma^{\prime\prime}}^{-1}\mu^\prime&=(\sigma^{-2}-\sigma^{-4}x_\text{new}\Sigma^{\prime\prime}x_\text{new})y_\text{new}^2-2(\sigma^{-2}x_\text{new}\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}\mu)y_\text{new}+c
\end{aligned}$$

On retrouve donc :

$$\begin{aligned}
{\sigma^\prime}^{-2}=\sigma^{-2}(1-\sigma^{-2}x_\text{new}^T\Sigma^{\prime\prime}x_\text{new})\\
\mu^{\prime\prime}={\sigma^\prime}^2\sigma^{-2}x_\text{new}^T\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}\mu.
\end{aligned}$$

Nous avons :

$$\begin{aligned}
p(y_\text{new}|x_\text{new}, \tilde{X}, y)&\propto\text{exp}\Big(-\frac{{\sigma^\prime}^{-2}}{2}(y_\text{new}-\mu^{\prime\prime)^2}\Big)
\end{aligned}$$




#### Étape 2.3 : Un peu de nettoyage

Nous avons grâce à la formule de [Sherman–Morrison](https://en.wikipedia.org/wiki/Sherman–Morrison_formula) :

$$\Sigma^{\prime\prime}=(x_\text{new}x_\text{new}^T\sigma^{-2}+{\Sigma^\prime}^{-1})^{-1}={\Sigma^\prime}-\frac{{\Sigma^\prime}\sigma^{-2}x_\text{new}x_\text{new}^T{\Sigma^\prime}}{1+\sigma^{-2}x_\text{new}^T{\Sigma^\prime}x_\text{new}}$$

Nous avons donc :

$$x_\text{new}^T\Sigma^{\prime\prime}x_{\text{new}}=x_\text{new}^T\Sigma^\prime x_\text{new}-\frac{\sigma^{-2}(x_\text{new}^T\Sigma^\prime x_\text{new})^2}{1+\sigma^{-2}x_\text{new}^T{\Sigma^\prime}x_\text{new}}=\frac{x_\text{new}^T\Sigma^\prime x_\text{new}}{1+\sigma^{-2}x_\text{new}^T\Sigma^\prime x_\text{new}}.$$

Reprenons $\sigma^\prime$. Nous avons enfin :

$${\sigma^\prime}^{-2}=\sigma^{-2}(1-\sigma^{-2}\frac{x_\text{new}^T\Sigma^\prime x_\text{new}}{1+\sigma^{-2}x_\text{new}^T\Sigma^\prime x_\text{new}})=\frac{\sigma^{-2}}{1+\sigma^{-2}x_\text{new}^T\Sigma^\prime x_\text{new}}$$

Considérons maintenant $\mu^{\prime\prime}$ :

$$\mu^{\prime\prime}={\sigma^\prime}^2\sigma^{-2}x_\text{new}^T\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}\mu=\mu^T{\sigma^\prime}^2\sigma^{-2}\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}x_\text{new}.$$

Traitons la partie à droite et montrons :

$$\begin{aligned}
{\sigma^\prime}^2\sigma^{-2}\Sigma^{\prime\prime}{\Sigma^\prime}^{-1}x_\text{new}&=x_\text{new}\\
\Leftrightarrow {\sigma^\prime}^2\sigma^{-2} x_\text{new}&={\Sigma^{\prime\prime}}^{-1}\Sigma^\prime x_\text{new}\\&
=(x_\text{new}x_\text{new}^T\sigma^{-2}+{\Sigma^\prime}^{-1})\Sigma^\prime x_\text{new}\\
&=\sigma^{-2}x_\text{new}x_\text{new}^T\Sigma^\prime x_\text{new}+x_\text{new}\\
&= (\sigma^{-2}x_\text{new}^T\Sigma^\prime x_\text{new} + 1)x_\text{new}\\
\Leftrightarrow {\sigma^\prime}^2x_\text{new}&=\frac{\sigma^{-2}x_\text{new}^T\Sigma^\prime x_\text{new} + 1}{\sigma^{-2}}x_\text{new}\\
\Leftrightarrow x_\text{new}&=x_\text{new}.
\end{aligned}$$

Et nous obtenons ce que nous voulions montrer. Cela nous indique que nous avons : 

$$\mu^{\prime\prime}=\mu^T x_\text{new},$$

où 

$$\mu=(\tilde{X}^T\tilde{X}+\lambda I)^{-1}\tilde{X}^T y.$$

### 3. Conclusion

Nous obtenons donc, étant donné une observation $x_\text{new}$ que son label $y_\text{new}$ est distribué selon une loi normale dont la moyenne est :

$$\mu^T x_\text{new}$$

avec :

$$\mu=(\tilde{X}^T\tilde{X}+\lambda I)^{-1}\tilde{X}^T y,$$

et dont la variance est donnée par :

$${\sigma^\prime}^{-2}=\frac{\sigma^{-2}}{1+\sigma^{-2}x_\text{new}^T\Sigma^\prime x_\text{new}}$$

où 

$${\Sigma^\prime}^{-1}=\frac{\tilde{X}^T\tilde{X}+\lambda I}{\sigma^2}.$$

Cela nous dit finalement qu'une prédiction d'un modèlre Ridge revient à prendre l'espérance d'un Bayesian Model Averaging où nos modèles sont les modèles linéaires auquel on ajoute comme *prior* une loi normale dont la variance est contrôlée par le paramètre $1/\lambda$.

On retrouve ce modèle dans $\texttt{sklearn}$ avec $\texttt{BayesianRidge}$.

---

## III. Pratique

**<span style='color:blue'> Exercice</span>** 
**Dans le code ci-dessous, jouez avec le *prior* de la régression linéaire Bayésienne pour en voir les effets.**



 ----

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import BayesianRidge


def func(x): return np.sin(2*np.pi*x)


# ############
# Generate sinusoidal data with noise
size = 25
rng = np.random.RandomState(1234)
x_train = rng.uniform(0., 1., size)
y_train = func(x_train) + rng.normal(scale=0.1, size=size)
x_test = np.linspace(0., 1., 100)


# ############
# Fit by cubic polynomial
n_order = 3
X_train = np.vander(x_train, n_order + 1, increasing=True)
X_test = np.vander(x_test, n_order + 1, increasing=True)

# ############
# Plot the true and predicted curves with log marginal likelihood (L)
reg = BayesianRidge(tol=1e-6, fit_intercept=False, compute_score=True)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
for i, ax in enumerate(axes):
    # Bayesian ridge regression with different initial value pairs
    if i == 0:
        init = [1 / np.var(y_train), 1.]  # Default values
    elif i == 1:
        ####### Complete this part ######## or die ####################
        init = [1., 1e-3]
        reg.set_params(alpha_init=init[0], lambda_init=init[1])
        ###############################################################
    reg.fit(X_train, y_train)
    ymean, ystd = reg.predict(X_test, return_std=True)

    ax.plot(x_test, func(x_test), color="blue", label="sin($2\\pi x$)")
    ax.scatter(x_train, y_train, s=50, alpha=0.5, label="observation")
    ax.plot(x_test, ymean, color="red", label="predict mean")
    ax.fill_between(x_test, ymean-ystd, ymean+ystd,
                    color="pink", alpha=0.5, label="predict std")
    ax.set_ylim(-1.3, 1.3)
    ax.legend()
    title = "$\\alpha$_init$={:.2f},\\ \\lambda$_init$={}$".format(
            init[0], init[1])
    if i == 0:
        title += " (Default)"
    ax.set_title(title, fontsize=12)
    text = "$\\alpha={:.1f}$\n$\\lambda={:.3f}$\n$L={:.1f}$".format(
           reg.alpha_, reg.lambda_, reg.scores_[-1])
    ax.text(0.05, -1.0, text, fontsize=12)

plt.tight_layout()
plt.show()